In [10]:
import os
import cv2
import shutil
import base64
import requests
import numpy as np
import mediapipe as mp
os.makedirs("output", exist_ok=True)

FORGE_INSTANCE_URL = "https://3v5yzyzlt689rs-1234.proxy.runpod.net" # "http://127.0.0.1:1234" / "https://*-1234.proxy.runpod.net"

In [11]:


def imagen_img2img(img_inp_path, img_msk_path=None, img_out_path="_test.jpg"):

    img_inp_base64 = None
    img_msk_base64 = None
    if img_inp_path:
        with open(img_inp_path, "rb") as f:
            img_inp_base64 = base64.b64encode(f.read()).decode('utf-8')
    if img_msk_path:
        with open(img_msk_path, "rb") as f:
            img_msk_base64 = base64.b64encode(f.read()).decode('utf-8')
    img_h, img_w, _ = cv2.imread(img_inp_path).shape

    sd_params = {
        "prompt": "masterpiece, best quality, 1girl, artist: bacun",
        "width": img_w,
        "height": img_h,
        "model": "WAI_V11",
        # "model": "MJX_V07",
        "steps": 20,
        "cfg": 5,
        "denoising": 0.5,
    }

    payload = {
        "prompt": sd_params["prompt"],
        "negative_prompt": "bad quality, worst quality, worst detail, artist name, text, signature, logo, watermark",
        "styles": [],
        "seed": -1,
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": "Euler a",
        "scheduler": "Karras",
        "batch_size": 1,
        "n_iter": 1,
        "steps": sd_params["steps"],
        "cfg_scale": sd_params["cfg"],
        "distilled_cfg_scale": 3.5,
        "width": sd_params["width"],
        "height": sd_params["height"],
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "eta": 0,
        "denoising_strength": sd_params["denoising"],
        "s_min_uncond": 0.0,
        "s_churn": 0.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "s_noise": 1.0,
        "override_settings": {"sd_model_checkpoint": sd_params["model"], "CLIP_stop_at_last_layers": 1},
        "override_settings_restore_afterwards": False,
        # "refiner_checkpoint": "string",
        "refiner_switch_at": 0,
        "disable_extra_networks": False,
        # "firstpass_image": "string",
        "comments": {},
        "init_images": [img_inp_base64],
        "resize_mode": 1,
        "image_cfg_scale": 1.5,
        "mask": img_msk_base64,
        "mask_blur_x": 4,
        "mask_blur_y": 4,
        "mask_blur": 4,
        "mask_round": True,
        "inpainting_fill": 1,
        "inpaint_full_res": 0,
        "inpaint_full_res_padding": 32,
        "inpainting_mask_invert": 0,
        "initial_noise_multiplier": 1.0,
        # "latent_mask": "string",
        # "force_task_id": "string",
        "hr_distilled_cfg": 3.5,
        "sampler_index": "Euler",
        "include_init_images": False,
        "script_name": None,
        "script_args": [],
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {},
        # "infotext": "string"
    }

    def request_forge_instance(payload, savepath):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/img2img", json=payload) as req:
                with open(savepath, "wb") as f:
                    f.write(base64.b64decode(req.json()["images"][0]))
        except Exception as e:
            print(f"⚠️ Error: {e}")
            print(f"⚠️ req.json(): {req.json()}")

    request_forge_instance(payload, img_out_path)

In [12]:
img_inp_path = "input/00.jpg"
img_msk_path = None # "input/00_mask.jpg"
img_out_path = "_test.jpg"

imagen_img2img(img_inp_path, img_msk_path, img_out_path)